In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("combined_unique.csv", encoding='latin1')

C:\Users\Minal\AppData\Local\Temp\ipykernel_5928\1658328909.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("combined_unique.csv", encoding='latin1')


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ Drop rows with missing Customer ID or Description
df_cf = df.dropna(subset=['Customer ID', 'Description'])

# 2️⃣ Handle any non-numeric Customer IDs
# Convert to string → keep only numeric rows
df_cf = df_cf[df_cf['Customer ID'].apply(lambda x: str(x).replace('.', '').isdigit())]

# Convert to integer safely
df_cf['Customer ID'] = df_cf['Customer ID'].astype(int)

# 3️⃣ Create User–Item matrix (Customer × Product)
user_item_matrix = (
    df_cf.groupby(['Customer ID', 'Description'])['Quantity']
    .sum()
    .unstack()
    .fillna(0)
)

# 4️⃣ Convert to binary (implicit purchase info)
user_item_matrix = (user_item_matrix > 0).astype(int)

# 5️⃣ Compute item–item cosine similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

# 6️⃣ Function: Recommend top-N items for a specific user
def recommend_items_for_user(customer_id, n=5):
    if customer_id not in user_item_matrix.index:
        print(f"Customer {customer_id} not found in data.")
        return None

    # Products the user already bought
    user_purchases = user_item_matrix.loc[customer_id]
    purchased_items = user_purchases[user_purchases > 0].index.tolist()

    # Compute recommendation scores
    recommendation_scores = item_similarity_df[purchased_items].mean(axis=1)

    # Remove items the user already purchased
    recommendation_scores = recommendation_scores.drop(labels=purchased_items, errors='ignore')

    # Top-N recommendations
    top_recommendations = recommendation_scores.sort_values(ascending=False).head(n)

    return top_recommendations

# Example: Recommend top 5 products for a sample customer
sample_customer = user_item_matrix.index[0]  # or manually specify, e.g. 17850
print(f"\n🛒 Top 5 recommended products for Customer {sample_customer}:")
print(recommend_items_for_user(sample_customer, n=5))



🛒 Top 5 recommended products for Customer 12346:
Description
DOORMAT HEARTS                   0.246826
DOORMAT RED RETROSPOT            0.245667
DOORMAT ENGLISH ROSE             0.220740
DOORMAT KEEP CALM AND COME IN    0.219500
DOORMAT TOPIARY                  0.209062
dtype: float64


In [4]:
# Example: Recommend top 5 products for a sample customer
sample_customer = user_item_matrix.index[1000]  # or manually specify, e.g. 17850
print(f"\n🛒 Top 5 recommended products for Customer {sample_customer}:")
print(recommend_items_for_user(sample_customer, n=5))



🛒 Top 5 recommended products for Customer 13355:
Description
LUNCH BAG PINK POLKADOT          0.323782
JUMBO BAG RED RETROSPOT          0.318160
LUNCH BAG SUKI DESIGN            0.301724
LUNCH BAG DOLLY GIRL DESIGN      0.301110
LUNCH BAG VINTAGE LEAF DESIGN    0.297513
dtype: float64


In [5]:
# Example: check if a specific Customer ID exists
customer_to_test = 12931  # replace with any valid ID from your dataset

if customer_to_test in user_item_matrix.index:
    print(f"🛒 Top 5 recommended products for Customer {customer_to_test}:")
    print(recommend_items_for_user(int(customer_to_test), n=5))
else:
    print(f"⚠️ Customer {customer_to_test} not found in dataset.")


🛒 Top 5 recommended products for Customer 12931:
Description
LUNCH BAG SPACEBOY DESIGN     0.219294
LUNCH BAG  BLACK SKULL.       0.215384
JUMBO BAG RED RETROSPOT       0.212237
LUNCH BAG RED RETROSPOT       0.212024
LUNCH BAG CARS BLUE           0.209525
dtype: float64
